In [53]:
import pandas as pd
import json
import boto3
import datetime
import os
import numpy as np
import re 
from urllib import request
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import io
import collections
import contextlib
import sys
import wave
import math
import webrtcvad
from scipy.io import wavfile
import requests
from azure.ai.anomalydetector import AnomalyDetectorClient
from azure.ai.anomalydetector.models import DetectRequest, TimeSeriesPoint, TimeGranularity, \
    AnomalyDetectorError
from azure.core.credentials import AzureKeyCredential
import pandas as pd
from dateutil import parser
from pymongo import MongoClient


import time

SUBSCRIPTION_KEY = '20fb2829f07c4018938ce3a45fc0bcd1'
ANOMALY_DETECTOR_ENDPOINT = 'https://v-dat.cognitiveservices.azure.com/anomalydetector/v1.1-preview/timeseries/entire/detect'



class VDAT():
    
    def __init__(self):
        self.df = None
        self.vrTime = None
        self.eyeTrack = None
        self.pHmd = None
        self.pEyetracker = None
        self.pBvp = None
        self.pEda = None
        self.pTmp = None
        self.pIbi = None
        self.pHr = None
        self.requestPosX = None
        self.requestPosY = None
        self.requestPosZ = None
        self.requestCombinedX = None
        self.requestCombinedY = None
        self.requestCombinedZ = None
        self.requestBvp = None
        self.requestEda = None
        self.requestTmp = None
        self.requestBvp = None
        self.requestIbi = None
        self.requestHr = None
        
        
            
    def csv2df(self, csv):
    
        data = pd.read_csv(csv)
        data = data[data[' Start']==1].reset_index(drop=True)
        data = data.fillna(0)
#         data = data.reset_index(drop=True)
        data['time'] = [i.replace(" ", "") for i in data[' Time']]
        data['unix'] = [time.mktime(datetime.datetime.strptime(str(data['Date'][i])+data['time'][i],'%Y%m%d%H:%M:%S').timetuple())
                            for i in range(0, len(data))]
        datatime = data[['Date', 'time']]
        vrDate = datatime['Date'][0]
        vrTime = []
        for i in sorted(datatime['time']):
            t = datetime.datetime.strptime(str(vrDate)+ ' ' + i, "%Y%m%d %H:%M:%S")
            vrTime.append(t.isoformat())
        data['vrTime'] = vrTime
        
        data = data.drop(' Time', axis=1)
        
        return data
    

    def getVolume(self, link):
    
        # # path from URL
        # url = "https://dt1amnyxy57si.cloudfront.net/audios/0317_Audio.wav"
        # Freq = 16000
        # context = ssl._create_unverified_context()
        # audio_bytes = request.urlopen(url,context=context).read()
    
        #path from local
        audio_bytes = wave.open(link,'rb')
        Freq = 16000
        audio_bytes = audio_bytes.readframes(14400000)
        
        # Convert wav to audio_segment
        audio_segment = AudioSegment.from_raw(
            io.BytesIO(audio_bytes),
            sample_width=2,
            frame_rate=Freq,
            channels=1
        )
    
        nonsilent_data = detect_nonsilent(
            audio_segment,
            min_silence_len=600,
            silence_thresh=-37,
            seek_step=1
        )
    
        remove_list = []
        for i in nonsilent_data:
            if ((i[1] - i[0]) < 350):
                remove_list.append(i)
        for i in remove_list:
            nonsilent_data.remove(i)
    
    
        header = audio_bytes[:70]
    
        user_talk_time = nonsilent_data
        
        time_list = list(range(0,math.ceil(nonsilent_data[-1][-1]/1000)))
    
        
        time_per_1s = []
        for i in range(len(time_list)-1):
            time_per_1s.append([time_list[i]*1000,time_list[i+1]*1000])
          
        audio_seg_u = []
    
        for i in time_per_1s:
            a = i[0] * 32 + 16000
            b = i[1] * 32 + 16000
            audio_seg_u.append(AudioSegment.from_raw(
              io.BytesIO(header + audio_bytes[a:b]),
              sample_width=2,
              frame_rate=Freq,
              channels=1
            ))
    
    
        volumes = []
        for i in range(len(audio_seg_u)):
            volumes.append(audio_seg_u[i].dBFS)
    
    
        return volumes
                    
    def processE4(self, data):
        
        idx = data['idx']
        value = []
        iso = []   
        xxx = []

        for i in idx:
            temp = [float(i) for i in data[data['idx'] == i]['value']]
            value.append(round(sum(temp) / len(temp),4))
            t = float(i)
            iso.append(datetime.datetime.fromtimestamp(t).isoformat())
            xxx.append(datetime.datetime.fromtimestamp(t).isoformat()[14:19])
                    
        pData = pd.DataFrame()
        pData['idx'] = idx
        pData['iso'] = iso
        pData['temp'] = xxx
        pData['value'] = value
        
        pattern = re.compile(r':\d\d')
        sec = []        
        for i in pData['iso']:
            sec.append(re.findall(pattern,i)[1][1:]) 
        pData['sec'] = sec
        
        mValue = []
        xxx = sorted(list(set(xxx)))
        for i in xxx:     
            xxxx = pData[pData['temp'] == i]['value']
            mValue.append(xxxx.mean())
        
        requestData = pd.DataFrame()
        requestData['timestamp'] = sorted(list(set([i[:19] for i in pData['iso']])))  
        requestData['value'] = mValue
        
#         sensorStart = int(requestData['timestamp'][0][-2:])
#         sensorEnd = int(requestData['timestamp'][len(requestData)-1][-2:])
        
#         temp = pd.DataFrame(columns=['timestamp', 'value'])
    
#         if sensorEnd > end:
#             requestData = requestData[:-(sensorEnd - end)]
#         if sensorEnd <= end:
#             t = []
#             v = []
#             for i in range(1, end - sensorEnd +1):
#                 t.append(datetime.datetime.strptime(requestData['timestamp'][len(requestData)-1], "%Y-%m-%dT%H:%M:%S") + datetime.timedelta(seconds=i))
#                 v.append(requestData['value'].mean())
#             temp['timestamp'] = t
#             temp['value'] = v
#             requestData = requestData.append(temp).reset_index(drop=True)
        
            
#         temp = pd.DataFrame(columns=['timestamp', 'value'])
#         t = []
#         v = []
#         for i in range(0, sensorStart - start):
#             t.append(datetime.datetime.strptime(vrTime['vrTime'][0], "%Y-%m-%dT%H:%M:%S") + datetime.timedelta(seconds=i))
#             v.append(requestData['value'].mean())
#         temp['timestamp'] = t
#         temp['value'] = v
        
#         requestData = temp.append(requestData).reset_index(drop=True)
        
        pData = pData[['idx', 'iso', 'sec', 'value']]
    
        return pData, requestData
    
    def processAnother(self, data, vrTime):
        idx = vrTime['idx']
        pData = pd.DataFrame(columns=[data.columns])
        timeList = []
        sec = []
        for i in list(set(idx)):
            t_data = data[data['idx'] == i]
            sec.append(list(vrTime[data['idx'] == i]['vrTime'])[0][-2:])
            timeList.append(list(vrTime[data['idx'] == i]['vrTime'])[0])
            pData.loc[len(pData)] = list(t_data.mean())
        pData['timestamp'] = timeList
        pData['sec'] = sec    
        
        return pData
    
    def makeRequestHmd(self, data):
        
        requestPosX = pd.DataFrame()
        requestPosY = pd.DataFrame()
        requestPosZ = pd.DataFrame()
        
        requestPosX['timestamp'] = data['timestamp']
        requestPosX['value'] =  data[' Left_pos.x']
        
        requestPosY['timestamp'] = data['timestamp']
        requestPosY['value'] =  data[' Left_pos.y']
        
        requestPosZ['timestamp'] = data['timestamp']
        requestPosZ['value'] =  data[' Left_pos.z']
        
        return requestPosX, requestPosY, requestPosZ
    
    def makeRequestEyetracker(self, data):
        requestCombinedX = pd.DataFrame()
        requestCombinedY = pd.DataFrame()
        requestCombinedZ = pd.DataFrame()
        
        requestCombinedX['timestamp'] = data['timestamp']
        requestCombinedX['value'] = data[' combined_x']
    
        requestCombinedY['timestamp'] = data['timestamp']
        requestCombinedY['value'] = data[' combined_y']
    
        requestCombinedZ['timestamp'] = data['timestamp']
        requestCombinedZ['value'] = data[' combined_z']
        
        return requestCombinedX, requestCombinedY, requestCombinedZ
    
    
    def makeJson(self, data):
        p = {'granularity': 'secondly'}
        t = []
        for i, v in enumerate(data['timestamp']):
            t.append({'timestamp': str(v),'value': data['value'][i]})
        p['series'] = t
        
        return p
    
    
    def json_default(self, value): 
        if isinstance(value, datetime.date): 
            return value.strftime('%Y-%m-%d') 
        raise TypeError('not JSON serializable')
    
    
    def detect(self, endpoint, apikey, request_data):
        headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': apikey}
        response = requests.post(endpoint, data=json.dumps(request_data, default=self.json_default), headers=headers)
        if response.status_code == 200:
            return json.loads(response.content.decode("utf-8"))
        else:
            print(response.status_code)
            raise Exception(response.text)
    
    
    def getAnomalyResult(self, data):
        result = self.detect(ANOMALY_DETECTOR_ENDPOINT, SUBSCRIPTION_KEY, data)
        result = {'expectedValues': result['expectedValues'], 'isAnomaly': result['isAnomaly'], 'isNegativeAnomaly': result['isNegativeAnomaly'],
              'isPositiveAnomaly': result['isPositiveAnomaly'], 'upperMargins': result['upperMargins'], 'lowerMargins': result['lowerMargins'],
              'timestamp': [x['timestamp'] for x in data['series']], 
              'value': [x['value'] for x in data['series']]}
    
        return result, result['isAnomaly']
    
    def makeChunk(self, data):
        chunk = []
        for i in range(0, len(data)-1):
            n = data[i]
            if n+1 < data[i+1]:
                chunk.append(data.index(n))
            elif i+2 == len(data):
                chunk.append(data.index(n))            
                
        n = 0    
        
        chunks = []
        for i in range(0, len(chunk)):
            chunks.append(data[n:chunk[i]+1])
            n = chunk[i] +1
            
        return chunks    
    
    def getSensorResult(self, sensor=None, voice=None):
    #     client = MongoClient(host="localhost", port=27017)
    #     db = client['v-dat']
    #     sensor = db['sensor']
        
        data = self.csv2df(sensor)        

        if voice is not None:
            vo = self.getVolume(voice)
            volume = pd.DataFrame(columns=['idx','timestamp','value'])
            volume['idx'] = [i for i in range(0, len(vo))]
            volume['timestamp'] = [datetime.datetime.fromtimestamp(i).isoformat() for i in volume['idx']]
            volume['value'] = [v for i, v in enumerate(vo)]
            sendVolume = self.makeJson(volume)
            anomalyVolume, volumePoint = self.getAnomalyResult(sendVolume) 
            
        incol = []
        for i in data['unix']:
            incol.append(i - data['unix'][0])
        data['idx'] = incol
        self.df = data.copy()
    
        self.vrTime = data[['idx','vrTime']]
        
        eyeTracking = []
        vrTracking = []
        for i in sorted(list(set(data['idx']))):
            temp = [float(i) for i in data[data['idx'] == i][' PlayerSee']]
            if 1 in temp:
                eyeTracking.append(1)
            elif 2 in temp:       
                eyeTracking.append(2)                
            elif 1 not in temp:
                eyeTracking.append(0)
                

        for i in sorted(list(set(data['idx']))):
            temp = [float(i) for i in data[data['idx'] == i][' HaveToSee']]
            if 1 in temp:
                vrTracking.append(1)
            elif 2 in temp:       
                vrTracking.append(2)                
            elif 1 not in temp:
                vrTracking.append(0)

    
                
        
#         hmd = data[['idx','unix', ' Left_pos.x', ' Left_pos.y', ' Left_pos.z',
#                            ' Left_rot.x', ' Left_rot.y', ' Left_rot.z']]
        e4Eda = data[[' EDA']]
        e4Bvp = data[[' BVP']]
        e4Tmp = data[[' TMP']]
        e4Ibi = data[[' IBI']]
        
        
#         eyeTracker = data[['idx','unix', ' combined_x', ' combined_y', ' combined_z', ' r_bit', ' r_open',
#                        ' r_dir_x', ' r_dir_y', ' r_dir_z', ' r_gaze_origin_x',
#                        ' r_gaze_origin_y', ' r_gaze_origin_z', ' r_pupil', ' r_pupil_pos',
#                        ' r_pupil_pos_x', ' r_pupil_pos_y', ' r_frown', ' r_squeeze',
#                        ' r_widel_bit', ' l_open', ' l_dir_x', ' l_dir_y', ' l_dir_z',
#                        ' l_gaze_origin_x', ' l_gaze_origin_y', ' l_gaze_origin_z', ' l_pupil',
#                        ' l_pupil_pos', ' l_pupil_pos_x', ' l_pupil_pos_y', ' l_frown',
#                        ' l_squeeze', ' l_wide']]
    
    
        e4Bvp = pd.DataFrame(data= [i[8:].split() for i in e4Bvp[' BVP']], columns=['unix','value'])
        e4Eda = pd.DataFrame(data= [i[8:].split() for i in e4Eda[' EDA']], columns=['unix','value'])
        e4Tmp = pd.DataFrame(data= [i[16:].split() for i in e4Tmp[' TMP']], columns=['unix','value'])        
        e4Ibi = pd.DataFrame(data= [i[8:].split() for i in e4Ibi[' IBI']], columns=['unix','value'])  
        
        e4Bvp['idx'] = incol
        e4Eda['idx'] = incol
        e4Tmp['idx'] = incol
        e4Ibi['idx'] = incol

        e4Hr = e4Ibi.copy()
        a = np.full(len(e4Hr['value']), 60)
        e4Hr['value'] = a / np.array(e4Hr['value'], dtype=float)
    
#         self.pHmd = self.processAnother(hmd, self.vrTime)
#         self.pHmd = self.pHmd[['idx','unix','timestamp', 'sec', ' Left_pos.x', ' Left_pos.y', ' Left_pos.z', 
#                  ' Left_rot.x', ' Left_rot.y',' Left_rot.z']]
        
    
    
#         self.pEyetracker = self.processAnother(eyeTracker,self.vrTime)
#         self.pEyetracker = self.pEyetracker[['idx', 'unix','timestamp', 'sec', ' combined_x', ' combined_y', ' combined_z',
#                                ' r_bit', ' r_open', ' r_dir_x', ' r_dir_y', ' r_dir_z', ' r_gaze_origin_x', 
#                                ' r_gaze_origin_y',' r_gaze_origin_z',' r_pupil',' r_pupil_pos',' r_pupil_pos_x',
#                                ' r_pupil_pos_y', ' r_frown', ' r_squeeze', ' r_widel_bit', ' l_open',
#                                ' l_dir_x', ' l_dir_y', ' l_dir_z', ' l_gaze_origin_x', ' l_gaze_origin_y',
#                                ' l_gaze_origin_z', ' l_pupil', ' l_pupil_pos', ' l_pupil_pos_x', ' l_pupil_pos_y',
#                                ' l_frown',' l_squeeze', ' l_wide']]
       
            
#         self.requestPosX, self.requestPosY, self.requestPosZ = self.makeRequestHmd(self.pHmd)
#         self.requestCombinedX, self.requestCombinedY, self.requestCombinedZ = self.makeRequestEyetracker(self.pEyetracker)
    
        self.pBvp, self.requestBvp = self.processE4(e4Bvp)
        self.pEda, self.requestEda = self.processE4(e4Eda)
        self.pTmp, self.requestTmp = self.processE4(e4Tmp)
        self.pIbi, self.requestIbi = self.processE4(e4Ibi)
        self.pHr, self.requestHr = self.processE4(e4Hr)
    
#         sendPosX = self.makeJson(self.requestPosX)
#         sendPosY = self.makeJson(self.requestPosY)
#         sendPosZ = self.makeJson(self.requestPosZ)
#         sendCombinedX = self.makeJson(self.requestCombinedX)
#         sendCombinedY = self.makeJson(self.requestCombinedY)
#         sendCombinedZ = self.makeJson(self.requestCombinedZ)
    
        sendBvp = self.makeJson(self.requestBvp)
        sendEda = self.makeJson(self.requestEda)
        sendTmp = self.makeJson(self.requestTmp)
        sendIbi = self.makeJson(self.requestIbi)
        sendHr = self.makeJson(self.requestHr)

    
#         anomalyPosX = self.getAnomalyResult(sendPosX)
#         anomalyPosY = self.getAnomalyResult(sendPosY)
#         anomalyPosZ = self.getAnomalyResult(sendPosZ)
#         anomalyCombinedX = self.getAnomalyResult(sendCombinedX)
#         anomalyCombinedY = self.getAnomalyResult(sendCombinedY)
#         anomalyCombinedZ = self.getAnomalyResult(sendCombinedZ)
        
#         anomalyVolume, volumePoint = self.getAnomalyResult(sendVolume)
        anomalyBvp, bvpPoint = self.getAnomalyResult(sendBvp)
        anomalyEda, edaPoint = self.getAnomalyResult(sendEda)
        anomalyTmp, tmpPoint = self.getAnomalyResult(sendTmp)
        anomalyIbi, ibiPoint = self.getAnomalyResult(sendIbi)        
        anomalyHr, hrPoint = self.getAnomalyResult(sendHr)
        
        
        bvpPoint = [i for i,v in enumerate(bvpPoint) if v ==True]
        edaPoint = [i for i,v in enumerate(edaPoint) if v ==True]
        tmpPoint = [i for i,v in enumerate(tmpPoint) if v ==True]
        ibiPoint = [i for i,v in enumerate(ibiPoint) if v ==True]
        hrPoint = [i for i,v in enumerate(hrPoint) if v ==True]
#         bvpPoint = [i for i,v in enumerate(bvpPoint) if v ==True]
        
        anomalyPoints = sorted(list(set([*bvpPoint, *edaPoint,*tmpPoint,*ibiPoint,*hrPoint,])))
        
        
        vrCoworker = [i for i,v in enumerate(vrTracking) if v == 2]
        vrGuest = [i for i,v in enumerate(vrTracking) if v == 1]
        vrNone = [i for i,v in enumerate(vrTracking) if v == 0]

        eyeCoworker = [i for i,v in enumerate(eyeTracking) if v == 2]
        eyeGuest = [i for i,v in enumerate(eyeTracking) if v == 1]
        eyeNone = [i for i,v in enumerate(eyeTracking) if v == 0]
        
        vrCoworker = self.makeChunk(vrCoworker)
        vrGuest = self.makeChunk(vrGuest)
        vrNone = self.makeChunk(vrNone)        
        vrAll = [vrCoworker, vrGuest, vrNone]
        
        eyeCoworker = self.makeChunk(eyeCoworker)
        eyeGuest = self.makeChunk(eyeGuest)
        eyeNone = self.makeChunk(eyeNone)
        
        eyeCoworker = [[i[0],i[-1]] for i in  eyeCoworker]        
        eyeGuest = [[i[0],i[-1]] for i in  eyeGuest]
        
        eyeAll = [eyeCoworker, eyeGuest]

#         return data
        return {'eye': eyeAll, 'vr': vrAll, 'bvp': anomalyBvp, 'eda': anomalyEda, 'volume': anomalyVolume,
                'tmp': anomalyTmp, 'ibi': anomalyIbi, 'hr': anomalyHr ,'anomaly': anomalyPoints}
    


In [54]:
a = VDAT()

In [55]:
sensor = "20220328_160153_data(head,e4,eye).csv"
voice = '20220331_180000_voice.wav'
data = a.getSensorResult(sensor, voice)        

In [58]:
data['volume']['value']

[-52.03272165152002,
 -16.639052345598127,
 -18.22880005094975,
 -76.32959861247397,
 -80.76657360480111,
 -90.30899869919435,
 -90.30899869919435,
 -90.30899869919435,
 -90.30899869919435,
 -90.30899869919435,
 -90.30899869919435,
 -23.219146687405626,
 -18.588511051454844,
 -30.50222160344233,
 -78.2677988726351,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -90.30899869919435,
 -80.76657360480111,
 -90.30899869919435,
 -90.30899869919435,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -21.811365966573018,
 -19.319124434894096,
 -22.850758639792225,
 -76.32959861247397,
 -80.76657360480111,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -80.76657360480111,
 -84.28839878591472,
 -84.28839878591472,
 -84.28839878591472,
 -18.973

In [45]:
data['eye']

[[6, 7, 8, 9, 10, 11, 12, 13],
 [16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
 [30, 31, 32, 33, 34, 35],
 [39, 40, 41, 42, 43],
 [48, 49],
 [58, 59, 60, 61, 62, 63, 64, 65],
 [69, 70],
 [82, 83, 84],
 [95, 96, 97, 98, 99, 100, 101, 102, 103, 104],
 [115, 116, 117, 118, 119, 120, 121, 122, 123],
 [125, 126, 127, 128, 129, 130],
 [135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]]

In [25]:
eyeTracking = []
vrTracking = []
for i in sorted(list(set(data['idx']))):
    temp = [float(i) for i in data[data['idx'] == i][' PlayerSee']]
    if 1 in temp:
        eyeTracking.append(1)
    elif 2 in temp:       
        eyeTracking.append(2)                
    elif 1 not in temp:
        eyeTracking.append(0)
    
for i in sorted(list(set(data['idx']))):
    temp = [float(i) for i in data[data['idx'] == i][' HaveToSee']]
    if 1 in temp:
        vrTracking.append(1)
    elif 2 in temp:       
        vrTracking.append(2)                
    elif 1 not in temp:
        vrTracking.append(0)

In [26]:
vrCoworker = [i for i,v in enumerate(vrTracking) if v == 2]
vrGuest = [i for i,v in enumerate(vrTracking) if v == 1]
vrNone = [i for i,v in enumerate(vrTracking) if v == 0]
eyeCoworker = [i for i,v in enumerate(eyeTracking) if v == 2]
eyeGuest = [i for i,v in enumerate(eyeTracking) if v == 1]
eyeNone = [i for i,v in enumerate(eyeTracking) if v == 0]


In [40]:
[i for i,v in enumerate(eyeTracking) if v == 2]

[74, 75, 76, 77, 78, 79, 80]

In [79]:
def makeChunk(data):
    chunk = []
    for i in range(0, len(data)-1):
        n = data[i]
        if n+1 < data[i+1]:
            chunk.append(data.index(n))
        elif i+2 == len(data):
            chunk.append(data.index(n))            
    n = 0    
    
    chunks = []
    for i in range(0, len(chunk)):
        chunks.append(data[n:chunk[i]+1])
        n = chunk[i] +1
        
    return chunks

In [86]:
no = makeChunk(vrCoworker)

In [87]:
no

[[72, 73, 74, 75, 76, 77, 78]]

In [21]:

eyeTracking = []
vrTracking = []

for i in sorted(list(set(df['idx']))):
    temp = [float(i) for i in df[df['idx'] == i][' PlayerSee']]
    print(temp)
    if 1 in temp:
        print(1)
        eyeTracking.append(1)
    elif 2 in temp:
        print(2)
        eyeTracking.append(2)
    elif 1 not in temp:
        print(0)
        eyeTracking.append(0)
    print('\n')

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
0


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [12]:
eyeTracking

[0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0]

In [152]:
from urllib import request
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import io
import collections
import contextlib
import sys
import wave
import math
import webrtcvad
from scipy.io import wavfile

def voice_py():

    # # path from URL
    # url = "https://dt1amnyxy57si.cloudfront.net/audios/0317_Audio.wav"
    # Freq = 16000
    # context = ssl._create_unverified_context()
    # audio_bytes = request.urlopen(url,context=context).read()

    #path from local
    audio_bytes = wave.open("./20220308_test/20220310_124417_voice.wav",'rb')
    Freq = 16000
    audio_bytes = audio_bytes.readframes(14400000)
    
    # Convert wav to audio_segment
    audio_segment = AudioSegment.from_raw(
        io.BytesIO(audio_bytes),
        sample_width=2,
        frame_rate=Freq,
        channels=1
    )

    nonsilent_data = detect_nonsilent(
        audio_segment,
        min_silence_len=600,
        silence_thresh=-37,
        seek_step=1
    )

    remove_list = []
    for i in nonsilent_data:
        if ((i[1] - i[0]) < 350):
            remove_list.append(i)
    for i in remove_list:
        nonsilent_data.remove(i)


    header = audio_bytes[:70]

    user_talk_time = nonsilent_data
    
    time_list = list(range(0,math.ceil(nonsilent_data[-1][-1]/1000)))

    
    time_per_1s = []
    for i in range(len(time_list)-1):
        time_per_1s.append([time_list[i]*1000,time_list[i+1]*1000])
      
    audio_seg_u = []

    for i in time_per_1s:
        a = i[0] * 32 + 16000
        b = i[1] * 32 + 16000
        audio_seg_u.append(AudioSegment.from_raw(
          io.BytesIO(header + audio_bytes[a:b]),
          sample_width=2,
          frame_rate=Freq,
          channels=1
        ))


    volumes = []
    for i in range(len(audio_seg_u)):
        volumes.append(audio_seg_u[i].dBFS)


    return volumes
    # plt.plot(volumes)

In [153]:
v = voice_py()

In [154]:
volume = pd.DataFrame(columns=['idx','timestamp','value'])
volume['idx'] = [i for i in range(0, len(v))]
volume['timestamp'] = [datetime.datetime.fromtimestamp(i).isoformat() for i in volume['idx']]
volume['value'] = [v for i, v in enumerate(v)]

In [155]:
volume

,idx,timestamp,value
0,0,1970-01-01T09:00:00,-61.365838
1,1,1970-01-01T09:00:01,-65.203549
2,2,1970-01-01T09:00:02,-66.787174
3,3,1970-01-01T09:00:03,-66.226599
4,4,1970-01-01T09:00:04,-61.061039
...,...,...,...
360,360,1970-01-01T09:06:00,-21.982847
361,361,1970-01-01T09:06:01,-19.440055
362,362,1970-01-01T09:06:02,-62.009532
363,363,1970-01-01T09:06:03,-22.980540


In [145]:
a.getAnomalyResult(a.makeJson(volume))

{'expectedValues': [-71.22414851040786,
  -69.62714008173273,
  -70.15947622462444,
  -70.4256442960703,
  -70.40231517669511,
  -70.40231517669511,
  -71.04111854816516,
  -70.85808858592246,
  -70.67505862367976,
  -70.67505862367977,
  -70.49202866143708,
  -70.14342795856149,
  -69.97785721792857,
  -69.81228647729566,
  -69.74132758845299,
  -69.57575684782007,
  -69.64671573666274,
  -69.71767462550542,
  -69.77286487238307,
  -66.10582995625145,
  -65.6868885541714,
  -65.61592966532871,
  -65.89357147936161,
  -66.18698193535957,
  -70.10800570257652,
  -70.19966239823493,
  -69.83146710351062,
  -59.48461790556339,
  -49.484648360327625,
  -39.75348359295549,
  -30.601207387940484,
  -27.20055680346845,
  -33.05307483142415,
  -42.414241005189886,
  -51.796805069686435,
  -61.62496668399867,
  -65.55938330382796,
  -70.0537144738195,
  -70.50948788304686,
  -70.6750586236798,
  -70.6750586236798,
  -70.85808858592245,
  -70.85808858592245,
  -71.0411185481651,
  -71.0411185481